## File names here:

In [1]:
tissues_with_atac = ['CHO','Brain','Liver','Lung','Kidney','Spleen','BMDMwt','BMDM1hKLA']

## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']
tss_annotation = doc['tss_annotation']
mRNA_peak_file = doc["mRNA_peak_file"]

In [2]:
ls {tss_annotation}

/data/isshamie/genome/ncbi_anno_103/start_site_mRNA_final.tsv


In [3]:
from create_output import *

txn_f = "Results/tss_annotation/txn_df_02_tissues.p"
#expr_f = "Results/merged/samples.merge.peaksexpression.log10"
expr_f = "Results/merged/tissues.merge.peaksexpression.log10"
anno_f = "Results/tss_annotation_peaks/all_peaks_txn_df_02_maxinfo.tsv"
rank_func = rank_median_samples
atac_f = "Results/ATAC_results/CHO/ATAC_ppr.naive_overlap.narrowPeak.sort"
tss_f = mRNA_peak_file
all_atac = "Results/ATAC_results/ATAC_merge.bed"



In [4]:

import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 13)


In [5]:
if not os.path.exists("Results/output"):
    os.mkdir("Results/output")

In [6]:
%%time
# bed_df, meta_df = construct_peakID(txn_anno_f, expr_f, anno_f, rank_func,
#                      atac_f=None, col_name=None)

bed_df, meta_df = construct_all_peaks(txn_f, expr_f, anno_f, rank_func, tss_f, atac_f,
                        all_atac=all_atac, out_f="Results/output/TSS1")

Constructing TSS not observed experimentally


10637it [01:56, 91.64it/s]


Constructing TSS observed experimentally


0it [00:00, ?it/s]

Including ATAC information


48344it [09:06, 88.52it/s]
48344it [25:49, 31.21it/s]


CPU times: user 52min 49s, sys: 11.8 s, total: 53min 1s
Wall time: 52min 41s


In [7]:
bed_df.head()

,Chr,Start,End,ID,Stat,Strand
p1@Bmp4_XM_027386528.1,NW_020822366.1,724533,724684,p1@Bmp4_XM_027386528.1,4.074290,-
p1@Cdkn3_XM_027386531.1,NW_020822366.1,1085431,1085582,p1@Cdkn3_XM_027386531.1,1.212188,+
p0@Cdkn3_XM_027386533.1,NW_020822366.1,1085743,1085745,p0@Cdkn3_XM_027386533.1,0.000000,+
p2@Cnih1_XM_027386534.1,NW_020822366.1,1113842,1113993,p2@Cnih1_XM_027386534.1,1.049218,-
p1@Cnih1_XM_027386534.1,NW_020822366.1,1114280,1114431,p1@Cnih1_XM_027386534.1,1.527630,-


In [8]:
meta_df.head()

,Tissues,cs,CHO ATAC Region,ID,Gene,Gene ID,Transcript,Is Experimental,has ATAC
p1@Bmp4_XM_027386528.1,"Heart,Spleen,MiscOrgans,FemaleReproductive,Bra...",0b11,,308710,Bmp4,,XM_027386528.1,1,1
p1@Cdkn3_XM_027386531.1,"BMDMwt,Spleen,MiscOrgans,FemaleReproductive,CH...",0b11,,237253,Cdkn3,,XM_027386531.1,1,0
p0@Cdkn3_XM_027386533.1,,,,,Cdkn3,,XM_027386533.1,0,0
p2@Cnih1_XM_027386534.1,"Heart,Spleen,MiscOrgans,FemaleReproductive,CHO...",0b11,1113861:1114585,353645,Cnih1,,XM_027386534.1,1,1
p1@Cnih1_XM_027386534.1,"Heart,Spleen,MiscOrgans,FemaleReproductive,CHO...",0b11,,298307,Cnih1,,XM_027386534.1,1,0


-----------------------------

In [6]:
txn_df = pickle.load(open(txn_f,"rb"))
anno_df = pd.read_csv(anno_f, sep="\t", index_col=0)

In [7]:
txn_df.head()

,peaks,closest_to_X_numberOfPeaks,bin_-1000_1000,sameStrand_bin_-1000_1000,samples,minDistance,minDistancePeak,maxValue,maxPeakId,maxSample,Chr,Start,End,Strand,gene,transcript,maxSamples,hasGene,Number of SS,Tissues
XM_027386528.1,"[1394, 3038, 17698, 17726, 21888, 24414, 35067...",131,"[163408, 308710]",[308710],"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",27.000000,308710,11864.6,308710,NaN,NW_020822366.1,724442,724725,-,Bmp4,XM_027386528.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,1,"Heart,Spleen,MiscOrgans,FemaleReproductive,Bra..."
XM_027386531.1,"[17679, 17707, 83014, 89129, 99005, 108459, 10...",29,"[138244, 237253]",[237253],"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",74.000000,237253,15.3,237253,NaN,NW_020822366.1,1085406,1085613,+,Cdkn3,XM_027386531.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,1,"BMDMwt,Spleen,MiscOrgans,FemaleReproductive,CH..."
XM_027386533.1,"[68904, 137124, 249586, 270147, 335001, 400304...",8,[],[],[],inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],False,0,
XM_027386534.1,"[14157, 68988, 69200, 116089, 132667, 167473, ...",19,"[132667, 298307, 350581, 353645]","[298307, 353645]","[MiscOrgans_START1_and_MiscOrgans_startinput1,...",88.000000,298307,57.2,298307,NaN,NW_020822366.1,1114184,1114467,-,Cnih1,XM_027386534.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,2,"Heart,Spleen,MiscOrgans,FemaleReproductive,CHO..."
XM_027386535.1,"[34817, 35013, 35070, 86845, 97343, 120275, 12...",16,"[35013, 97343, 120275, 122677, 161643, 268973,...","[97343, 122677]","[MiscOrgans_START1_and_MiscOrgans_startinput1,...",291.000000,122677,146.8,122677,NaN,NW_020822366.1,1152433,1153082,-,Gmfb,XM_027386535.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,2,"Heart,Spleen,MiscOrgans,FemaleReproductive,CHO..."


In [8]:
anno_df.head()

,Chr,Start,End,Strand,Stat,Parent files,Total subpeaks,f04_peaks/MiscOrgans_START1_and_MiscOrgans_startinput1,f04_peaks/Liver_START2_and_Liver_startinput1,f04_peaks/Brain_GROCap1_and_Brain_GRO1,...,max_Chr,max_Start,max_End,max_Stat,cho_file,cho_file_peak_name,cho_Chr,cho_Start,cho_End,cho_Stat
ID,,,,,,,,,,,,,,,,,,,,,
4,NW_020822601.1,37427816,37427999,+,129.031738,f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...,23,NW_020822601.1-18,NaN,NW_020822601.1-1445,...,NW_020822601.1,37427849,37427999,457.8,f04_peaks/CHO_START1_and_CHO_startinput1,NW_020822601.1-41,NW_020822601.1,37427849,37427999,91.90
5,NW_020822619.1,2231719,2231926,-,16.203333,f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...,24,NW_020822619.1-12,NaN,NW_020822619.1-53,...,NW_020822619.1,2231771,2231921,61.3,NaN,NaN,NaN,-1,-1,0.00
6,NW_020822577.1,6314085,6314339,+,11.444000,f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...,25,NW_020822577.1-68,NW_020822577.1-6,NaN,...,NW_020822577.1,6314179,6314329,77.8,f04_peaks/CHO_START1_and_CHO_startinput1,NW_020822577.1-193,NW_020822577.1,6314096,6314246,6.12
8,NW_020822435.1,461540,462075,-,6.744815,f04_peaks/MiscOrgans_START1_and_MiscOrgans_sta...,27,NW_020822435.1-25,NaN,NW_020822435.1-42,...,NW_020822435.1,461768,461918,15.0,f04_peaks/CHO_START1_and_CHO_startinput1,NW_020822435.1-23,NW_020822435.1,461773,461923,10.70
12,NW_020822644.1,1117287,1117499,-,3.882334,f04_peaks/Liver_START2_and_Liver_startinput1|f...,6,NaN,NW_020822644.1-33,NaN,...,NW_020822644.1,1117287,1117437,10.6,NaN,NaN,NaN,-1,-1,0.00


In [12]:
txn = pickle.load(open(txn_f, "rb"))
txn

,peaks,closest_to_X_numberOfPeaks,bin_-1000_1000,sameStrand_bin_-1000_1000,samples,minDistance,minDistancePeak,maxValue,maxPeakId,maxSample,Chr,Start,End,Strand,gene,transcript,maxSamples,hasGene,Number of SS,Tissues
XM_027386528.1,"[1394, 3038, 17698, 17726, 21888, 24414, 35067...",131,"[163408, 308710]",[308710],"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",27.000000,308710,11864.6,308710,NaN,NW_020822366.1,724442,724725,-,Bmp4,XM_027386528.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,1,"Heart,Spleen,MiscOrgans,FemaleReproductive,Bra..."
XM_027386531.1,"[17679, 17707, 83014, 89129, 99005, 108459, 10...",29,"[138244, 237253]",[237253],"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",74.000000,237253,15.3,237253,NaN,NW_020822366.1,1085406,1085613,+,Cdkn3,XM_027386531.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,1,"BMDMwt,Spleen,MiscOrgans,FemaleReproductive,CH..."
XM_027386533.1,"[68904, 137124, 249586, 270147, 335001, 400304...",8,[],[],[],inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],False,0,
XM_027386534.1,"[14157, 68988, 69200, 116089, 132667, 167473, ...",19,"[132667, 298307, 350581, 353645]","[298307, 353645]","[MiscOrgans_START1_and_MiscOrgans_startinput1,...",88.000000,298307,57.2,298307,NaN,NW_020822366.1,1114184,1114467,-,Cnih1,XM_027386534.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,2,"Heart,Spleen,MiscOrgans,FemaleReproductive,CHO..."
XM_027386535.1,"[34817, 35013, 35070, 86845, 97343, 120275, 12...",16,"[35013, 97343, 120275, 122677, 161643, 268973,...","[97343, 122677]","[MiscOrgans_START1_and_MiscOrgans_startinput1,...",291.000000,122677,146.8,122677,NaN,NW_020822366.1,1152433,1153082,-,Gmfb,XM_027386535.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,2,"Heart,Spleen,MiscOrgans,FemaleReproductive,CHO..."
XM_027386536.1,"[5407, 35121, 224603, 293332, 306506, 338794, ...",10,"[5407, 293332, 306506, 348288, 414132]","[348288, 414132]","[MiscOrgans_START1_and_MiscOrgans_startinput1,...",68.000000,348288,144.9,348288,NaN,NW_020822366.1,1159824,1160078,+,Cgrrf1,XM_027386536.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,2,"Heart,Spleen,MiscOrgans,FemaleReproductive,CHO..."
XM_027386537.1,"[108474, 141481, 154640, 310411, 342596, 444116]",6,"[310411, 342596]",[342596],"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",-18.000000,342596,21.2,342596,NaN,NW_020822366.1,1214715,1214931,+,Samd4a,XM_027386537.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,1,"Heart,Spleen,MiscOrgans,FemaleReproductive,Bra..."
XM_027386538.1,"[68906, 69090, 69201, 71536, 71578, 122006, 13...",39,"[165659, 226267, 309749]",[226267],[Brain_GROCap1_and_Brain_GRO1],428.000000,226267,0.793,226267,NaN,NW_020822366.1,1215196,1215346,+,Samd4a,XM_027386538.1,[Brain_GROCap1_and_Brain_GRO1],True,1,Brain
XM_027386539.1,"[7315, 12014, 14154, 14156, 35119, 69060, 7148...",73,"[334958, 479018, 509869]","[334958, 509869]","[MiscOrgans_START1_and_MiscOrgans_startinput1,...",-130.000000,334958,14.8,509869,NaN,NW_020822366.1,1332777,1332927,+,Samd4a,XM_027386539.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,2,"MiscOrgans,FemaleReproductive,Muscle"
XM_027386540.1,"[10177, 10874, 17708, 69091, 71579, 99009, 990...",34,"[154630, 238436, 243948]",[243948],"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",9.000000,243948,51.1,243948,NaN,NW_020822366.1,1513696,1513877,-,Gch1,XM_027386540.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,1,"Heart,BMDMwt,Spleen,MiscOrgans,FemaleReproduct..."


In [14]:
tss_f

'/data/isshamie/genome/ncbi_anno_103/mRNA_final.peak'

In [15]:
doc['tss_annotation']

'/data/isshamie/genome/ncbi_anno_103/start_site_mRNA_final.tsv'

In [ ]:
tss_df = pd.read_csv(tss_file, sep='\t', index_col=0)
tss_df[tss_df_col].unique()